In [9]:
import json
from pathlib import Path
import re

In [10]:
benchmark = ["bt", "cg", "ep", "ft", "is", "lu", "mg", "sp"]
size = "B"
workdir = Path("/home/studyztp/test_ground/experiments/hardware-profiling/NPB_protocol/NPB3.4.2/NPB3.4-OMP")

In [11]:
def get_static_info(path):
    info = {}
    with open(path, "r") as f:
        line = f.readline()
        line = f.readline()
        while line:
            match = re.match(r'\[(\d+):(\w+)\] \[(\d+):(.*)\] \[(\d+)\]', line.strip())
            if match:
                function_id, function_name, basic_block_id, basic_block_name, basic_block_count = match.groups()
                if function_id not in info:
                    info[function_id] = {
                        "function_name": function_name,
                        "basic_blocks": {}
                    }
                info[function_id]["basic_blocks"][basic_block_id] = {
                    "basic_block_name": basic_block_name,
                    "basic_block_count": basic_block_count
                }
            line = f.readline()
    return info

def form_dict_from_bbv(line):
    line = line.split()
    formed_dict = {}
    for item in line:
        if int(item.split(":")[1]) == 0:
            continue
        formed_dict[int(item.split(":")[0])] = int(item.split(":")[1])
    return formed_dict

def get_profiling_info(path):
    info = {}
    with open(path, "r") as f:
        line = f.readline()
        line = f.readline()
        info["roi_begin"] = form_dict_from_bbv(line)
        while line:
            match = re.match(r'Region(\d+)', line.strip())
            if match:
                region_id = match.groups()[0]
                info[region_id] = {
                    "inst" : int(line.split()[2])
                }
                line = f.readline()
                line = f.readline()
                info[region_id]["bbv"] = form_dict_from_bbv(line)
                line = f.readline()
                line = f.readline()
                info[region_id]["timestamp"] = form_dict_from_bbv(line)
            line = f.readline()
    return info


In [12]:
raw_profiling_info = {}
for bench in benchmark:
    bench_dir = Path(workdir/bench.upper())
    profiler_dir = Path(bench_dir/"profiling")
    static_info = get_static_info(Path(profiler_dir/"basicBlockInfo.txt"))
    profiling_info = get_profiling_info(Path(profiler_dir/"profiler_output.txt"))
    raw_profiling_info[bench] = {
        "static_info": static_info,
        "profiling_info": profiling_info
    }

with open("raw_profiling_info.json", "w") as f:
    json.dump(raw_profiling_info, f, indent=4)